# Baroque Ceiling Painters - Career Analysis

This notebook analyzes individual painter careers using the DuckDB database.

## Features
- **Painter CV**: Timeline of works, locations, and subjects
- **Geographic Footprint**: Where painters worked across German states
- **Thematic Analysis**: Common subjects and iconographic themes
- **Career Timeline**: Chronological view of commissions

In [ ]:
# =============================================================================
# Setup & Connect to Database
# =============================================================================
import duckdb
import polars as pl
import altair as alt
from pathlib import Path
from IPython.display import display as ipython_display, HTML, Markdown

# Configuration
DUCKDB_PATH = Path(r"c:/Users/thano/Documents/_Studium/KIT/DataStories/DataStories/baroque.duckdb") #change

# Enable Altair for larger datasets
alt.data_transformers.disable_max_rows()

# Connect to database
if DUCKDB_PATH.exists():
    con = duckdb.connect(str(DUCKDB_PATH), read_only=True)
    print(f"✅ Connected to: {DUCKDB_PATH.name}")
    
    # Quick stats
    n_paintings = con.execute("SELECT COUNT(*) FROM paintings").fetchone()[0]
    n_painters = con.execute("SELECT COUNT(DISTINCT person_id) FROM painting_persons WHERE role = 'PAINTER'").fetchone()[0]
    print(f"📊 Database contains {n_paintings} paintings by {n_painters} unique painters")
else:
    print(f"❌ Database not found: {DUCKDB_PATH}")
    print("   Run DataStory_Baroque_DuckDB.ipynb first to create the database.")

✅ Connected to: baroque.duckdb
📊 Database contains 4594 paintings by 332 unique painters


In [3]:
# =============================================================================
# List All Painters with Statistics
# =============================================================================

print("🎨 BAROQUE CEILING PAINTERS OVERVIEW")
print("=" * 70)

painters_overview = con.execute("""
    SELECT 
        pp.person_name,
        pp.person_id,
        COUNT(DISTINCT pp.nfdi_uri) as painting_count,
        COUNT(DISTINCT p.building_id) as building_count,
        COUNT(DISTINCT p.location_state) as states_count,
        MIN(p.year_start) as career_start,
        MAX(p.year_end) as career_end,
        CAST(MAX(p.year_end) - MIN(p.year_start) AS INTEGER) as career_span
    FROM painting_persons pp
    JOIN paintings p ON pp.nfdi_uri = p.nfdi_uri
    WHERE pp.role = 'PAINTER'
    GROUP BY pp.person_name, pp.person_id
    ORDER BY painting_count DESC, career_span DESC
""").pl()

print(f"\n📋 Found {len(painters_overview)} painters in database:\n")
print(painters_overview)

# Store painter list for selection
PAINTER_LIST = painters_overview['person_name'].to_list()

🎨 BAROQUE CEILING PAINTERS OVERVIEW

📋 Found 332 painters in database:

shape: (332, 8)
┌────────────┬────────────┬────────────┬───────────┬───────────┬───────────┬───────────┬───────────┐
│ person_nam ┆ person_id  ┆ painting_c ┆ building_ ┆ states_co ┆ career_st ┆ career_en ┆ career_sp │
│ e          ┆ ---        ┆ ount       ┆ count     ┆ unt       ┆ art       ┆ d         ┆ an        │
│ ---        ┆ str        ┆ ---        ┆ ---       ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│ str        ┆            ┆ i64        ┆ i64       ┆ i64       ┆ i32       ┆ i32       ┆ i32       │
╞════════════╪════════════╪════════════╪═══════════╪═══════════╪═══════════╪═══════════╪═══════════╡
│ Harms,     ┆ 5b74e880-6 ┆ 119        ┆ 7         ┆ 4         ┆ 1569      ┆ 1780      ┆ 211       │
│ Johann     ┆ e0d-11ea-a ┆            ┆           ┆           ┆           ┆           ┆           │
│ Oswald     ┆ 2e8-4df471 ┆            ┆           ┆           ┆           ┆           ┆           │
│  

In [4]:
# =============================================================================
# Painter CV Function
# =============================================================================

def generate_painter_cv(painter_name: str):
    """
    Generate a comprehensive CV for a baroque ceiling painter.
    
    Args:
        painter_name: Name of the painter (must match database)
    """
    
    # Get basic info
    basic_info = con.execute("""
        SELECT 
            pp.person_name,
            per.person_type,
            COUNT(DISTINCT pp.nfdi_uri) as total_works,
            COUNT(DISTINCT p.building_id) as buildings,
            COUNT(DISTINCT p.location_state) as states,
            MIN(p.year_start) as earliest_work,
            MAX(p.year_end) as latest_work
        FROM painting_persons pp
        JOIN paintings p ON pp.nfdi_uri = p.nfdi_uri
        LEFT JOIN persons per ON pp.person_id = per.person_id
        WHERE pp.role = 'PAINTER' AND pp.person_name = ?
        GROUP BY pp.person_name, per.person_type
    """, [painter_name]).fetchone()
    
    if not basic_info:
        print(f"❌ Painter '{painter_name}' not found in database.")
        print(f"\n💡 Available painters: {', '.join(PAINTER_LIST[:10])}...")
        return
    
    name, person_type, total_works, buildings, states, earliest, latest = basic_info
    
    # Header
    print("\n" + "=" * 70)
    print(f"🎨 PAINTER CV: {name}")
    print("=" * 70)
    
    # Basic Stats
    print(f"\n📋 OVERVIEW")
    print(f"   Type: {person_type or 'Individual'}")
    print(f"   Total documented works: {total_works}")
    print(f"   Buildings worked in: {buildings}")
    print(f"   German states covered: {states}")
    if earliest and latest:
        print(f"   Active period: {earliest} - {latest} ({latest - earliest} years)")
    
    # Works Timeline
    print(f"\n📅 WORKS TIMELINE")
    print("-" * 70)
    
    works = con.execute("""
        SELECT 
            p.label,
            p.year_start,
            p.year_end,
            p.year_is_approximate,
            p.building_name,
            p.building_function,
            p.room_name,
            p.location_state,
            p.nfdi_uri,
            p.imageUrl
        FROM painting_persons pp
        JOIN paintings p ON pp.nfdi_uri = p.nfdi_uri
        WHERE pp.role = 'PAINTER' AND pp.person_name = ?
        ORDER BY p.year_start NULLS LAST, p.building_name
    """, [painter_name]).pl()
    
    for i, row in enumerate(works.iter_rows(named=True), 1):
        year_str = ""
        if row['year_start']:
            approx = "~" if row['year_is_approximate'] else ""
            if row['year_end'] and row['year_end'] != row['year_start']:
                year_str = f"{approx}{row['year_start']}-{row['year_end']}"
            else:
                year_str = f"{approx}{row['year_start']}"
        else:
            year_str = "(undated)"
        
        building = row['building_name'] or "Unknown building"
        room = f" → {row['room_name']}" if row['room_name'] else ""
        state = f" [{row['location_state']}]" if row['location_state'] else ""
        func = f" ({row['building_function']})" if row['building_function'] else ""
        
        print(f"   {i:2}. [{year_str:>12}] {building[:40]}{func}")
        if room:
            print(f"                          Room: {row['room_name'][:50]}")
        print(f"                          State: {row['location_state'] or 'Unknown'}")
    
    # Geographic Distribution
    print(f"\n🗺️ GEOGRAPHIC FOOTPRINT")
    print("-" * 70)
    
    geo = con.execute("""
        SELECT 
            p.location_state,
            COUNT(*) as works,
            COUNT(DISTINCT p.building_id) as buildings
        FROM painting_persons pp
        JOIN paintings p ON pp.nfdi_uri = p.nfdi_uri
        WHERE pp.role = 'PAINTER' AND pp.person_name = ?
          AND p.location_state IS NOT NULL
        GROUP BY p.location_state
        ORDER BY works DESC
    """, [painter_name]).pl()
    
    for row in geo.iter_rows(named=True):
        print(f"   • {row['location_state']}: {row['works']} work(s) in {row['buildings']} building(s)")
    
    # Building Types
    print(f"\n🏛️ BUILDING TYPES")
    print("-" * 70)
    
    building_types = con.execute("""
        SELECT 
            p.building_function,
            COUNT(*) as works
        FROM painting_persons pp
        JOIN paintings p ON pp.nfdi_uri = p.nfdi_uri
        WHERE pp.role = 'PAINTER' AND pp.person_name = ?
          AND p.building_function IS NOT NULL
        GROUP BY p.building_function
        ORDER BY works DESC
    """, [painter_name]).pl()
    
    for row in building_types.iter_rows(named=True):
        print(f"   • {row['building_function']}: {row['works']} work(s)")
    
    # Subjects/Themes
    print(f"\n📚 ICONOGRAPHIC THEMES")
    print("-" * 70)
    
    subjects = con.execute("""
        SELECT 
            s.subject_label,
            s.subject_source,
            COUNT(*) as occurrences
        FROM painting_persons pp
        JOIN paintings p ON pp.nfdi_uri = p.nfdi_uri
        JOIN painting_subjects ps ON p.nfdi_uri = ps.nfdi_uri
        JOIN subjects s ON ps.subject_uri = s.subject_uri
        WHERE pp.role = 'PAINTER' AND pp.person_name = ?
          AND s.subject_label IS NOT NULL
        GROUP BY s.subject_label, s.subject_source
        ORDER BY occurrences DESC
        LIMIT 15
    """, [painter_name]).pl()
    
    if len(subjects) > 0:
        for row in subjects.iter_rows(named=True):
            source_tag = f"[{row['subject_source']}]" if row['subject_source'] else ""
            print(f"   • {row['subject_label'][:50]} {source_tag}: {row['occurrences']}x")
    else:
        print("   No subject data available for this painter.")
    
    # Collaborators (other painters on same works)
    print(f"\n🤝 COLLABORATORS")
    print("-" * 70)
    
    collaborators = con.execute("""
        SELECT 
            pp2.person_name as collaborator,
            pp2.role,
            COUNT(DISTINCT pp1.nfdi_uri) as shared_works
        FROM painting_persons pp1
        JOIN painting_persons pp2 ON pp1.nfdi_uri = pp2.nfdi_uri
        WHERE pp1.person_name = ? 
          AND pp2.person_name != ?
          AND pp1.role = 'PAINTER'
        GROUP BY pp2.person_name, pp2.role
        ORDER BY shared_works DESC
        LIMIT 10
    """, [painter_name, painter_name]).pl()
    
    if len(collaborators) > 0:
        for row in collaborators.iter_rows(named=True):
            print(f"   • {row['collaborator']} ({row['role']}): {row['shared_works']} shared work(s)")
    else:
        print("   No documented collaborators.")
    
    print("\n" + "=" * 70)
    
    return works  # Return works DataFrame for visualization

print("✅ Painter CV function ready.")
print(f"   Usage: generate_painter_cv('Painter Name')")
print(f"   Available painters: {len(PAINTER_LIST)}")

✅ Painter CV function ready.
   Usage: generate_painter_cv('Painter Name')
   Available painters: 332


In [5]:
# =============================================================================
# Select a Painter and Generate CV
# =============================================================================

# Change this to explore different painters!
SELECTED_PAINTER = PAINTER_LIST[0] if PAINTER_LIST else None

if SELECTED_PAINTER:
    works_df = generate_painter_cv(SELECTED_PAINTER)
else:
    print("No painters found in database.")


🎨 PAINTER CV: Harms, Johann Oswald

📋 OVERVIEW
   Type: ACTOR_PERSON
   Total documented works: 119
   Buildings worked in: 7
   German states covered: 4
   Active period: 1569 - 1780 (211 years)

📅 WORKS TIMELINE
----------------------------------------------------------------------
    1. [       ~1569] Weißenfels, Schloss Neu-Augustusburg
                          Room: Die Schlosskirche
                          State: Sachsen-Anhalt
    2. [        1682] Eisenberg, Residenzschloss Christiansbur
                          Room: Das Audienzzimmer der Herzogin (116)
                          State: Thüringen
    3. [        1682] Eisenberg, Residenzschloss Christiansbur
                          Room: Das Audienzzimmer der Herzogin (116)
                          State: Thüringen
    4. [        1682] Weißenfels, Schloss Neu-Augustusburg
                          Room: Die Schlosskirche
                          State: Sachsen-Anhalt
    5. [        1682] Weißenfels, Schloss Neu-Augu

In [6]:
# =============================================================================
# Painter Career Timeline Visualization
# =============================================================================

def visualize_painter_timeline(painter_name: str):
    """Create a timeline visualization of a painter's career."""
    
    timeline_data = con.execute("""
        SELECT 
            p.label,
            CAST(p.year_start AS INTEGER) as year_start,
            CAST(p.year_end AS INTEGER) as year_end,
            p.building_name,
            p.building_function,
            p.location_state
        FROM painting_persons pp
        JOIN paintings p ON pp.nfdi_uri = p.nfdi_uri
        WHERE pp.role = 'PAINTER' 
          AND pp.person_name = ?
          AND p.year_start IS NOT NULL
        ORDER BY p.year_start
    """, [painter_name]).pl()
    
    if len(timeline_data) == 0:
        print(f"No dated works found for {painter_name}")
        return
    
    # Convert to pandas for Altair
    df = timeline_data.to_pandas()
    df['year_end'] = df['year_end'].fillna(df['year_start'])
    df['duration'] = df['year_end'] - df['year_start'] + 1
    df['building_short'] = df['building_name'].str[:30] + '...'
    
    # Gantt-style timeline
    chart = alt.Chart(df).mark_bar().encode(
        x=alt.X('year_start:Q', title='Year', scale=alt.Scale(zero=False)),
        x2='year_end:Q',
        y=alt.Y('building_short:N', sort=alt.EncodingSortField('year_start'), title='Building'),
        color=alt.Color('building_function:N', title='Building Type'),
        tooltip=['label', 'building_name', 'building_function', 'location_state', 'year_start', 'year_end']
    ).properties(
        title=f'Career Timeline: {painter_name}',
        width=700,
        height=max(100, len(df) * 5)
    )
    
    ipython_display(chart)
    return chart

# Visualize selected painter
if SELECTED_PAINTER:
    print(f"\n📊 CAREER TIMELINE: {SELECTED_PAINTER}")
    visualize_painter_timeline(SELECTED_PAINTER)


📊 CAREER TIMELINE: Harms, Johann Oswald


alt.Chart(...)

In [7]:
# =============================================================================
# Painter Subject Preferences Visualization
# =============================================================================

def visualize_painter_subjects(painter_name: str):
    """Visualize the iconographic themes preferred by a painter."""
    
    subjects_data = con.execute("""
        SELECT 
            s.subject_label,
            s.subject_source,
            CAST(COUNT(*) AS INTEGER) as count
        FROM painting_persons pp
        JOIN paintings p ON pp.nfdi_uri = p.nfdi_uri
        JOIN painting_subjects ps ON p.nfdi_uri = ps.nfdi_uri
        JOIN subjects s ON ps.subject_uri = s.subject_uri
        WHERE pp.role = 'PAINTER' AND pp.person_name = ?
          AND s.subject_label IS NOT NULL
        GROUP BY s.subject_label, s.subject_source
        ORDER BY count DESC
        LIMIT 15
    """, [painter_name]).pl()
    
    if len(subjects_data) == 0:
        print(f"No subject data found for {painter_name}")
        return
    
    df = subjects_data.to_pandas()
    df['subject_short'] = df['subject_label'].str[:40]
    
    chart = alt.Chart(df).mark_bar().encode(
        x=alt.X('count:Q', title='Number of Works'),
        y=alt.Y('subject_short:N', sort='-x', title='Subject/Theme'),
        color=alt.Color('subject_source:N', title='Source'),
        tooltip=['subject_label', 'subject_source', 'count']
    ).properties(
        title=f'Iconographic Themes: {painter_name}',
        width=500,
        height=max(100, len(df) * 25)
    )
    
    ipython_display(chart)
    return chart

# Visualize selected painter's subjects
if SELECTED_PAINTER:
    print(f"\n📚 ICONOGRAPHIC THEMES: {SELECTED_PAINTER}")
    visualize_painter_subjects(SELECTED_PAINTER)


📚 ICONOGRAPHIC THEMES: Harms, Johann Oswald


alt.Chart(...)

In [8]:
# =============================================================================
# Compare Multiple Painters
# =============================================================================

def compare_painters(painter_names: list):
    """Compare career statistics of multiple painters."""
    
    if not painter_names:
        print("Please provide a list of painter names to compare.")
        return
    
    placeholders = ', '.join(['?' for _ in painter_names])
    
    comparison = con.execute(f"""
        SELECT 
            pp.person_name,
            CAST(COUNT(DISTINCT pp.nfdi_uri) AS INTEGER) as works,
            CAST(COUNT(DISTINCT p.building_id) AS INTEGER) as buildings,
            CAST(COUNT(DISTINCT p.location_state) AS INTEGER) as states,
            MIN(p.year_start) as career_start,
            MAX(p.year_end) as career_end
        FROM painting_persons pp
        JOIN paintings p ON pp.nfdi_uri = p.nfdi_uri
        WHERE pp.role = 'PAINTER' AND pp.person_name IN ({placeholders})
        GROUP BY pp.person_name
        ORDER BY works DESC
    """, painter_names).pl()
    
    print("\n🎨 PAINTER COMPARISON")
    print("=" * 70)
    print(comparison)
    
    # Visualization
    df = comparison.to_pandas()
    
    # Works comparison
    chart = alt.Chart(df).mark_bar().encode(
        x=alt.X('works:Q', title='Number of Works'),
        y=alt.Y('person_name:N', sort='-x', title='Painter'),
        color=alt.Color('states:Q', scale=alt.Scale(scheme='viridis'), title='States'),
        tooltip=['person_name', 'works', 'buildings', 'states', 'career_start', 'career_end']
    ).properties(
        title='Painter Comparison: Works & Geographic Reach',
        width=500,
        height=max(100, len(df) * 40)
    )
    
    ipython_display(chart)
    return comparison

# Compare top painters
if len(PAINTER_LIST) >= 2:
    top_painters = PAINTER_LIST[:min(5, len(PAINTER_LIST))]
    print(f"Comparing: {', '.join(top_painters)}")
    compare_painters(top_painters)

Comparing: Harms, Johann Oswald, Lammers, Seivert, Castelli, Carlo Ludovico, Asam, Cosmas Damian, Ritter, Johann Heinrich

🎨 PAINTER COMPARISON
shape: (5, 6)
┌──────────────────────────┬───────┬───────────┬────────┬──────────────┬────────────┐
│ person_name              ┆ works ┆ buildings ┆ states ┆ career_start ┆ career_end │
│ ---                      ┆ ---   ┆ ---       ┆ ---    ┆ ---          ┆ ---        │
│ str                      ┆ i32   ┆ i32       ┆ i32    ┆ i32          ┆ i32        │
╞══════════════════════════╪═══════╪═══════════╪════════╪══════════════╪════════════╡
│ Harms, Johann Oswald     ┆ 119   ┆ 7         ┆ 4      ┆ 1569         ┆ 1780       │
│ Lammers, Seivert         ┆ 82    ┆ 4         ┆ 1      ┆ 1670         ┆ 1750       │
│ Castelli, Carlo Ludovico ┆ 70    ┆ 8         ┆ 3      ┆ 1683         ┆ 1770       │
│ Asam, Cosmas Damian      ┆ 63    ┆ 7         ┆ 2      ┆ 1670         ┆ 1948       │
│ Ritter, Johann Heinrich  ┆ 58    ┆ 6         ┆ 1      ┆ 1580      

alt.Chart(...)

In [9]:
# =============================================================================
# Painter Geographic Map Data
# =============================================================================

def get_painter_locations(painter_name: str):
    """Get all locations where a painter worked with coordinates."""
    
    locations = con.execute("""
        SELECT 
            p.label,
            p.building_name,
            p.lat,
            p.lon,
            p.location_state,
            p.year_start,
            p.imageUrl
        FROM painting_persons pp
        JOIN paintings p ON pp.nfdi_uri = p.nfdi_uri
        WHERE pp.role = 'PAINTER' AND pp.person_name = ?
          AND p.lat IS NOT NULL AND p.lon IS NOT NULL
        ORDER BY p.year_start
    """, [painter_name]).pl()
    
    print(f"\n🗺️ WORK LOCATIONS: {painter_name}")
    print("=" * 70)
    print(f"Found {len(locations)} works with coordinates\n")
    
    if len(locations) > 0:
        # Create a simple point map
        df = locations.to_pandas()
        
        chart = alt.Chart(df).mark_circle(size=100).encode(
            longitude='lon:Q',
            latitude='lat:Q',
            color=alt.Color('location_state:N', title='State'),
            tooltip=['building_name', 'label', 'location_state', 'year_start']
        ).properties(
            title=f'Work Locations: {painter_name}',
            width=600,
            height=400
        ).project('mercator')
        
        ipython_display(chart)
    
    return locations

# Show locations for selected painter
if SELECTED_PAINTER:
    locations_df = get_painter_locations(SELECTED_PAINTER)


🗺️ WORK LOCATIONS: Harms, Johann Oswald
Found 119 works with coordinates



alt.Chart(...)

In [10]:
# =============================================================================
# Interactive Painter Explorer
# =============================================================================

# List all available painters for easy selection
print("🎨 AVAILABLE PAINTERS FOR ANALYSIS")
print("=" * 70)
print("Copy a painter name from below to explore their CV:\n")

for i, row in enumerate(painters_overview.iter_rows(named=True), 1):
    works = row['painting_count']
    buildings = row['building_count']
    career = f"{row['career_start']}-{row['career_end']}" if row['career_start'] else "undated"
    print(f"  {i:2}. {row['person_name']:<35} | {works:>2} works | {buildings:>2} buildings | {career}")

print("\n" + "=" * 70)
print("💡 To explore a specific painter, change SELECTED_PAINTER in cell 4")
print("   or call: generate_painter_cv('Painter Name')")

🎨 AVAILABLE PAINTERS FOR ANALYSIS
Copy a painter name from below to explore their CV:

   1. Harms, Johann Oswald                | 119 works |  7 buildings | 1569-1780
   2. Lammers, Seivert                    | 82 works |  4 buildings | 1670-1750
   3. Castelli, Carlo Ludovico            | 70 works |  8 buildings | 1683-1770
   4. Asam, Cosmas Damian                 | 63 works |  7 buildings | 1670-1948
   5. Ritter, Johann Heinrich             | 58 works |  6 buildings | 1580-1780
   6. Hermann, Franz Georg                | 54 works |  2 buildings | 1729-1742
   7. Kuen, Franz Martin                  | 40 works |  2 buildings | 1751-1758
   8. Paduano, Alessandro                 | 38 works |  1 buildings | 1500-1748
   9. Gödecke, Hans                       | 38 works |  1 buildings | 1595-1697
  10. Göding, Heinrich                    | 36 works |  3 buildings | 1559-1591
  11. Wolcker, Johann Georg               | 35 works |  1 buildings | 1739-1766
  12. Vaillant, Jacques         

In [11]:
# =============================================================================
# Explore Another Painter (Change name here!)
# =============================================================================

# 👇 CHANGE THIS NAME to explore a different painter
EXPLORE_PAINTER = PAINTER_LIST[1] if len(PAINTER_LIST) > 1 else PAINTER_LIST[0] if PAINTER_LIST else None

if EXPLORE_PAINTER:
    works = generate_painter_cv(EXPLORE_PAINTER)
    print("\n")
    visualize_painter_timeline(EXPLORE_PAINTER)
    visualize_painter_subjects(EXPLORE_PAINTER)


🎨 PAINTER CV: Lammers, Seivert

📋 OVERVIEW
   Type: ACTOR_PERSON
   Total documented works: 82
   Buildings worked in: 4
   German states covered: 1
   Active period: 1670 - 1750 (80 years)

📅 WORKS TIMELINE
----------------------------------------------------------------------
    1. [   1670-1680] Rudolstadt, Residenzschloss Heidecksburg
                          Room: Der Nebenraum zur so genannten Biedermeiergarderob
                          State: Thüringen
    2. [   1670-1680] Rudolstadt, Residenzschloss Heidecksburg
                          Room: Der Nebenraum zur so genannten Biedermeiergarderob
                          State: Thüringen
    3. [   1670-1680] Rudolstadt, Residenzschloss Heidecksburg
                          Room: Der Nebenraum zur so genannten Biedermeiergarderob
                          State: Thüringen
    4. [        1688] Leutenberg, ehem. Schloss "Friedensburg"
                          Room: Das so genannte Fürstenzimmer
                          St

alt.Chart(...)

alt.Chart(...)

In [12]:
# =============================================================================
# Close Connection (optional)
# =============================================================================

# Uncomment to close the connection when done
# con.close()
# print("✅ Database connection closed")

print("💡 Tip: This notebook connects in read-only mode.")
print("   The database can be used by multiple notebooks simultaneously.")

💡 Tip: This notebook connects in read-only mode.
   The database can be used by multiple notebooks simultaneously.
